In [34]:
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = "0d38f9d11cec49e3ad044a18de85df9b"
secret = "9a47fa59db3e4a42a70a8f79628ac454"
client_credentials_manager = SpotifyClientCredentials(client_id=id_, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [35]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,10000,50):
    track_results = sp.search(q='year:2019', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
      

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 61.06354408599998


In [36]:
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 10000


In [37]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

(10000, 4)


,artist_name,track_name,track_id,popularity
0,Travis Scott,HIGHEST IN THE ROOM,3eekarcy7kvN4yt5ZFzltW,99
1,Post Malone,Circles,21jGcNKet2qwijlDFuPiPb,100
2,Juice WRLD,Bandit (with YoungBoy Never Broke Again),6Gg1gjgKi2AK4e0qzsR7sd,94
3,Selena Gomez,Lose You To Love Me,1HfMVBKM75vxSfsQ5VefZ5,98
4,blackbear,hot girl bummer,7aiClxsDWFRQ0Kzk5KI5ku,96


In [39]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [40]:
# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [41]:
df_tracks.shape


(9409, 4)

In [42]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 164
Time to run this code (in seconds): 23.04280796699993


In [43]:
print('number of elements in the track_id list:', len(rows))


number of elements in the track_id list: 9245


In [44]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (9245, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.598,0.427,7,-8.764,0,0.0317,0.0546,0.000006,0.2100,0.0605,76.469,audio_features,3eekarcy7kvN4yt5ZFzltW,spotify:track:3eekarcy7kvN4yt5ZFzltW,https://api.spotify.com/v1/tracks/3eekarcy7kvN...,https://api.spotify.com/v1/audio-analysis/3eek...,175721,4
1,0.695,0.762,0,-3.497,1,0.0395,0.1920,0.002440,0.0863,0.5530,120.042,audio_features,21jGcNKet2qwijlDFuPiPb,spotify:track:21jGcNKet2qwijlDFuPiPb,https://api.spotify.com/v1/tracks/21jGcNKet2qw...,https://api.spotify.com/v1/audio-analysis/21jG...,215280,4
2,0.474,0.631,5,-5.884,0,0.3430,0.0687,0.000000,0.1320,0.4250,180.051,audio_features,6Gg1gjgKi2AK4e0qzsR7sd,spotify:track:6Gg1gjgKi2AK4e0qzsR7sd,https://api.spotify.com/v1/tracks/6Gg1gjgKi2AK...,https://api.spotify.com/v1/audio-analysis/6Gg1...,189323,4
3,0.505,0.340,4,-9.005,1,0.0438,0.5760,0.000000,0.2100,0.0916,101.993,audio_features,1HfMVBKM75vxSfsQ5VefZ5,spotify:track:1HfMVBKM75vxSfsQ5VefZ5,https://api.spotify.com/v1/tracks/1HfMVBKM75vx...,https://api.spotify.com/v1/audio-analysis/1HfM...,206459,4
4,0.778,0.559,6,-7.109,0,0.0776,0.1280,0.000000,0.3990,0.6780,129.989,audio_features,7aiClxsDWFRQ0Kzk5KI5ku,spotify:track:7aiClxsDWFRQ0Kzk5KI5ku,https://api.spotify.com/v1/tracks/7aiClxsDWFRQ...,https://api.spotify.com/v1/audio-analysis/7aiC...,185093,1


In [45]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(9245, 14)

In [46]:

# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (9245, 14)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Travis Scott,HIGHEST IN THE ROOM,3eekarcy7kvN4yt5ZFzltW,99,0.598,0.427,7,-8.764,0,0.0317,0.0546,0.000006,0.2100,0.0605,76.469,175721,4
1,Post Malone,Circles,21jGcNKet2qwijlDFuPiPb,100,0.695,0.762,0,-3.497,1,0.0395,0.1920,0.002440,0.0863,0.5530,120.042,215280,4
2,Juice WRLD,Bandit (with YoungBoy Never Broke Again),6Gg1gjgKi2AK4e0qzsR7sd,94,0.474,0.631,5,-5.884,0,0.3430,0.0687,0.000000,0.1320,0.4250,180.051,189323,4
3,Selena Gomez,Lose You To Love Me,1HfMVBKM75vxSfsQ5VefZ5,98,0.505,0.340,4,-9.005,1,0.0438,0.5760,0.000000,0.2100,0.0916,101.993,206459,4
4,blackbear,hot girl bummer,7aiClxsDWFRQ0Kzk5KI5ku,96,0.778,0.559,6,-7.109,0,0.0776,0.1280,0.000000,0.3990,0.6780,129.989,185093,1


In [47]:

df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


In [48]:
df.to_csv('SpotifyAudioFeatures2019.csv')